In [ ]:
!git clone https://github.com/danibachar/idc_CANLab.git

In [ ]:
!pip3 install -r ../requirements.txt

In [1]:
from google.colab import drive
from google.colab import files
    
from idc_CANLab.functions import utils,plots,consts, summary, backup # Our functionalities

from ipywidgets import interact, interactive, fixed, interact_manual#, FileUpload
import ipywidgets as widgets
from IPython.display import display
from IPython.display import FileLink, FileLinks
# Data manipulation and calculation packages
import pandas as pd
import numpy as np
import os

/usr/local/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


# Data Structures

In [2]:
dfs = {}
data_frame_names = []
cleaning_dfs = {}
cleaning_file_names = []

# Please enter your name, and a name for your experiment
## This will help us and you to build more meaningfull file names for your exported data and graphs

In [3]:
researcher_name = widgets.Text(
    value=utils.get_random_string(10),
    placeholder='Type your name'
)
project_name = widgets.Text(
    value=utils.get_random_string(6),
    placeholder='Type your project name'
)

names = widgets.VBox([researcher_name,project_name])
names

# Settings up environment variables to config your research environment

In [4]:
os.environ[consts.PROJECT_NAME_ENV_VAR]=project_name.value
os.environ[consts.USERNAME_ENV_VAR]=researcher_name.value

# 1) File Uploads
## Please note we currently support only CSV files.
##    If your'e using Excel or other file formats please convert to CSV
###  Excel to CSV - https://knowledgebase.constantcontact.com/articles/KnowledgeBase/6409-saving-an-excel-file-as-a-csv-file?lang=en_US
### EDF to CSV - https://emotiv.gitbook.io/emotivpro/convert_edf_to_csv

## 1.1) Choose Primary file to upload - Only one file!

## If you alerady uploaded the file you can select it 

In [5]:
def choose_filename(filename):
  return filename
primary_file_name_dropdown = widgets.Dropdown(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)
# primary_file_name_dropdown
interact(choose_filename, filename=primary_file_name_dropdown)

interactive(children=(Dropdown(description='File name:', options=('./idc_CANLab/examples/gal_data/bad_experime…

<function __main__.choose_filename(filename)>

In [6]:
file_name = primary_file_name_dropdown.value
dfs['primary'] = pd.read_csv(file_name)
data_frame_names = list(dfs.keys())

## If you have not uploaded the file already, please upload it
## If you used the 2 boxes above make sure to SKIP these 2 boxes, or else you will override your data

In [ ]:
uploaded = files.upload()
assert len(uploaded.items()) == 1

In [ ]:
for file_name, byte_file in uploaded.items():
  dfs['primary'] = pd.read_csv(io.StringIO(byte_file.decode("utf-8")))
data_frame_names = list(dfs.keys())

## 1.2) Choose Other files to upload, files we wish to merge to the primary file, like demographic data, metadata on the trial and others
## If you alerady uploaded the file you can select it 

In [7]:
def choose_filename(filename):
  return filename
secondary_files_name_dropdown = widgets.SelectMultiple(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)

interact(choose_filename, filename=secondary_files_name_dropdown)

interactive(children=(SelectMultiple(description='File name:', options=('./idc_CANLab/examples/gal_data/bad_ex…

<function __main__.choose_filename(filename)>

In [8]:
scondary_file_names = list(secondary_files_name_dropdown.value)
for file_name in scondary_file_names:
  dfs[file_name] = pd.read_csv(file_name)
data_frame_names = list(dfs.keys())

## If you have not uploaded the file already, please upload it
## If you used the 2 boxes above please skip these 2

In [ ]:
merged_uploaded = files.upload()

In [ ]:
for file_name, byte_file in merged_uploaded.items():
  data_frame_names.append(file_name)
  dfs[file_name] = pd.read_csv(io.StringIO(byte_file.decode("utf-8")))

## 1.3) Data Sumarization

In [9]:
print(summary.info_dfs(list(dfs.values())))

https://storage.googleapis.com/outliers/pyilrlsgiq/kytthc/info_07-18-2020-17%3A00%3A34.txt


# 2) Merging Extra data from external files - (for example memory span, age, etc)

### Please note that if you wish to look at the raw data before adding more from external sources, all you need to do is just skip the following box and move to the plotting part

### Please note, it is recommended to read a about JOIN and ANTI_JOIN opparations in SQL and Python Pandas to fully understand what we are doing here, although not mandatory. We would try and make the process as simple as possible


## 2.1) For each file we wish to merge, select the columns you wish to merge by with the primary file, these columns will be used as unique key and the recommendation will be the User Id/Session Label/Subject and Trial_Id/Trial Label.

In [10]:
def choose_colums(colums):
  return colums

# Widget that will be shwon at the bottom, after running the box
external_merge_columns_dropdown = []
for i in range(1,len(set(data_frame_names))):
  if data_frame_names[i] == "primary":
    continue
  w1 = widgets.SelectMultiple(
      options=dfs[data_frame_names[i]].columns,
      description=data_frame_names[i]
  )
  w2 = widgets.SelectMultiple(
      options=dfs[data_frame_names[0]].columns,
      description=data_frame_names[0]
  )
  external_merge_columns_dropdown.append(widgets.HBox([w1,w2]))
  
columns_map = widgets.VBox(external_merge_columns_dropdown)
columns_map

## 2.2) Merging is done automatically

In [11]:
merged_df = dfs[data_frame_names[0]].copy()
for i in range(1,len(set(data_frame_names))):
  child = columns_map.children[i-1]
  left = list(child.children[1].value)
  right = list(child.children[0].value)
  merged_df = merged_df.merge(dfs[data_frame_names[i]], left_on=left, right_on=right, how='inner')


## 2.3) Data Sumarization

In [12]:

print(summary.info_dfs(list(dfs.values())))
for sum_url in summary.describe_dfs(list(dfs.values())):
    print(sum_url)

https://storage.googleapis.com/outliers/pyilrlsgiq/kytthc/info_07-18-2020-17%3A01%3A03.txt
https://storage.googleapis.com/outliers/pyilrlsgiq/kytthc/descibe_07-18-2020-17%3A01%3A04.txt_object
https://storage.googleapis.com/outliers/pyilrlsgiq/kytthc/descibe_07-18-2020-17%3A01%3A04.txt_numeric
https://storage.googleapis.com/outliers/pyilrlsgiq/kytthc/descibe_07-18-2020-17%3A01%3A07.txt_numeric


# 3) Droping unwanted colums
## Here we suggest removing uneeded columns from our primary merged file, this could help both better performance and understanding of the data 


## 3.1) Choose unwanted colums

In [ ]:
def choose_colums(colums):
  return colums
# Widget that will be shwon at the bottom, after running the box
colums_dropdown = widgets.SelectMultiple(
    options=merged_df.columns,
    description='Select colums to drop:',
)

interact(choose_colums, colums=colums_dropdown)

## 3.2) Dropping the columns

In [ ]:
colums_to_drop = list(colums_dropdown.value)
merged_df = merged_df.drop(columns=colums_to_drop)

# Flattening 
for col in merged_df.columns:
  if merged_df[col].dtype == object:
    merged_df[col] = merged_df[col].astype('category')

## 3.3) Data Sumarization

In [ ]:
print(summary.info_dfs(list(dfs.values())))
print(summary.describe_dfs(list(dfs.values())))

## 3.4) Choose columns to convert to categorical, for example subject_id, trial_id, memory span and others should be categorical
## Wants to convert some numeric values for categorical? Do it now!

In [ ]:
def choose_colums(colums):
  return colums
# Widget that will be shwon at the bottom, after running the box
categorical_colums_dropdown = widgets.SelectMultiple(
    options=merged_df.columns,
    description='Select colums to drop:',
)

interact(choose_colums, colums=categorical_colums_dropdown)

In [ ]:
colums_to_convert_to_categorical = list(categorical_colums_dropdown.value)
for col in colums_to_convert_to_categorical:
  merged_df[col] = merged_df[col].astype('category')

# 4) Ploting the raw data we have so far

## In the next few boxes we are going through a process of plotting the trials a certain Subject was going through

## 4.1) Per Subject Ploting

### 4.1.1) Select the name of the column you wish to group by, usually the subject or trial columns

In [ ]:
def col_name(column_name):
  return column_name

groupped_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Subject:',
)

interact(col_name, column_name=groupped_column_dropdown)

## 4.1.2) Select the desire columns you wish to have as your y and x axis.
### For y, usually the column that follow the eye gaze on the Target, but could also be for the Competitor or one of the fillers
### For x, usually the time column

In [ ]:
def choose_axis(axis):
  return axis

y_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Select `Y` axis column name:',
)
x_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Select `X` axis column name:',
)

interact(choose_axis, axis=y_column_dropdown)
interact(choose_axis, axis=x_column_dropdown)

## 4.1.3) Select the different features (e.g memory span, critical, load, noise level) you want to create the grid by

In [ ]:
def choose_colums(colums):
  return colums
# Widget that will be shwon at the bottom, after running the box
features_column_dropdown = widgets.SelectMultiple(
    options=merged_df.columns,
    description='Select colums to drop:',
)

interact(choose_colums, colums=features_column_dropdown)

## 4.1.4) Plot!!

In [ ]:
groupping_col_name = groupped_column_dropdown.value
y_name = y_column_dropdown.value
x_name = x_column_dropdown.value
grid_features = list(features_column_dropdown.value)

_, url = plots.plots_by_group_and_features(merged_df, groupping_col_name, y_name, x_name, grid_features)
print("plots_by_group_and_features - ",url)
_, url = plots.plot_general_avg_grid(merged_df, y_name, x_name, grid_features)
print("plot_general_avg_grid - ",url)
_, url = plots.plot_general_avg(merged_df, y_name, x_name)
print("plot_general_avg - ",url)

# 5) Filtering and Data Cleaning
## You can repeat the folowing steps in order to clean different values from different colmuns

## 5.1) Drop rows with Nan/Null values

In [ ]:
merged_df = merged_df.dropna()

## 5.2) Choose Column to filter by

In [ ]:
def col_name(column_name):
  return column_name

filter_column_dropdown = widgets.Dropdown(
    options=merged_df.columns,
    description='Col Name:',
)

interact(col_name, column_name=filter_column_dropdown)

## 5.3) Choose value to remove from the data

In [ ]:
col_name = filter_column_dropdown.value
values = list(merged_df[col_name].unique())
def value_name(value):
  return value

value_name_dropdown = widgets.Dropdown(
    options=values,
    description='Value Name:',
)

interact(value_name, value=value_name_dropdown)

## 5.4) Filter that value out!

In [ ]:
value = value_name_dropdown.value
selector = (merged_df[col_name] != value)
merged_df = merged_df[selector]

## 5.5) Data Summarization and plotting

In [ ]:
print(summary.info_dfs(list(dfs.values())))
print(summary.describe_dfs(list(dfs.values())))

_, url = plots.plots_by_group_and_features(merged_df, y_name, x_name, grid_features)
print("plots_by_group_and_features - ",url)
_, url = plots.plot_general_avg_grid(merged_df, y_name, x_name, grid_features)
print("plot_general_avg_grid - ",url)
_, url = plots.plot_general_avg(merged_df, y_name, x_name)
print("plot_general_avg - ",url)

## 5.6) Filter according to exteral files.
## Here we will use the idea behind anti join to drop certain rows accroding to files you have prepared in advance

### 5.6.1) If you alerady uploaded the file you can select it

In [ ]:
def choose_filename(filename):
  return filename
anti_join_files_name_dropdown = widgets.SelectMultiple(
    options=utils.list_all_files_in_dir("."),
    description='File name:',
)

interact(choose_filename, filename=anti_join_files_name_dropdown)

In [ ]:
anti_join_file_names = list(anti_join_files_name_dropdown.value)
for file_name in anti_join_file_names:
  cleaning_file_names.append(file_name)
  cleaning_dfs[file_name] = pd.read_csv(file_name)

### 5.6.2) If you have not uploaded the file already, please upload it
### If you used the 2 boxes above please skip these 2

In [ ]:
anti_merge_uploaded = files.upload()

In [ ]:
for file_name, byte_file in anti_merge_uploaded.items():
  cleaning_file_names.append(file_name)
  cleaning_dfs[file_name] = pd.read_csv(io.StringIO(byte_file.decode("utf-8")))

### 5.6.3) Choose the unique column names you wish to anti-join.
### This columns hsould create a unique like key so we can tell our notebook to remove each row that corresponds from the file to the data we have cleaning and merged so far

In [ ]:
def choose_colums(colums):
  return colums

# Widget that will be shwon at the bottom, after running the box
external_clean_columns_dropdown = []
for i in range(0,len(cleaning_file_names)):
  w1 = widgets.SelectMultiple(
      options=cleaning_dfs[cleaning_file_names[i]].columns,
      description=cleaning_file_names[i]
  )
  w2 = widgets.SelectMultiple(
      options=merged_df.columns,
      description='primary data frame'
  )
  external_clean_columns_dropdown.append(widgets.HBox([w1,w2]))
  
columns_map = widgets.VBox(external_clean_columns_dropdown)
columns_map

In [ ]:
for i in range(len(cleaning_file_names)):
  child = columns_map.children[i-1]
  left = list(child.children[1].value)
  right = list(child.children[0].value)
  merged_df = anti_join(merged_df, cleaning_dfs[cleaning_file_names[i]], left_on=left, right_on=right)

### 5.6.4) Data Summarization and plotting

In [ ]:
print(summary.info_dfs(list(dfs.values())))
print(summary.describe_dfs(list(dfs.values())))

_, url = plots.plots_by_group_and_features(merged_df, y_name, x_name, grid_features)
print("plots_by_group_and_features - ",url)
_, url = plots.plot_general_avg_grid(merged_df, y_name, x_name, grid_features)
print("plot_general_avg_grid - ",url)
_, url = plots.plot_general_avg(merged_df, y_name, x_name)
print("plot_general_avg - ",url)

# 6) Saving files
## At ani stage you can choose to save the current state of our manipulated data

In [ ]:
urls = saved_file_name = save_locally_and_update(merged_df, dfs, cleaning_dfs)
for url in url:
    print(urls)

In [ ]:
_, url = plots.plots_by_group_and_features(merged_df, "RECORDING_SESSION_LABEL", "AVERAGE_IA_1_SAMPLE_COUNT_%", "BIN_START_TIME", ["condition","load"])
print("visit - ",url)
print("to watch the graphs")

In [ ]:
_, url = plots.plots_by_group_and_features(dfs['primary'], "RECORDING_SESSION_LABEL", "AVERAGE_IA_1_SAMPLE_COUNT_%", "BIN_START_TIME", ["condition","load"])
print("visit - ",url)
print("to watch the graphs")



In [ ]:
print(summary.info_dfs(list(dfs.values())))
print(summary.describe_dfs(list(dfs.values())))